In [141]:
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle
import transformers
import NERDA
import nltk

nltk.download('punkt')
from nltk import word_tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\99mor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [142]:
data_layer1 = pd.read_csv('data_layer1.csv')
data_layer2 = pd.read_csv('data_layer2.csv')

def exclude_low_freq_classes(data, threshold=10):
    df = data.copy()

    freq_table = df["pseudoLabels"].value_counts()
    low_freq_classes = []
    for i in range(len(freq_table)):
        if freq_table[i] < threshold:
            low_freq_classes.append(freq_table.index[i])

    for i in range(len(low_freq_classes)):
        df = df[df["pseudoLabels"] != low_freq_classes[i]]

    df.reset_index(drop=True, inplace=True)

    return df

data_layer1 = exclude_low_freq_classes(data_layer1, threshold=20)
data_layer2 = exclude_low_freq_classes(data_layer2, threshold=20)

C:\Users\99mor\.virtualenvs\MRP1_Models\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [143]:
def getuniquedomains(df, layer):
    unique_domains = df[layer].unique()
    unique_domains_dict = dict()
    for i, domain in enumerate(unique_domains):
        unique_domains_dict[domain] = i
    return unique_domains_dict

data_layer1_ddict = getuniquedomains(data_layer1, 'pseudoLabels')
data_layer2_ddict = getuniquedomains(data_layer2, 'pseudoLabels')

data_layer1['pseudoLabels'] = data_layer1['pseudoLabels'].map(lambda x: data_layer1_ddict[x])
data_layer2['pseudoLabels'] = data_layer2['pseudoLabels'].map(lambda x: data_layer2_ddict[x])

print(data_layer1_ddict)
print(data_layer2_ddict)

{'no skill': 0, 'business administration': 1, 'education, sales and marketing': 2, 'engineering, construction and transport': 3, 'information technology': 4, 'health and social care': 5, 'science and research': 6, 'Communication': 7, 'Time management': 8, 'Decision making': 9, 'Optimism': 10, 'Coaching': 11, 'Initiative': 12, 'Social skills': 13, 'Creativity': 14, 'Leadership': 15, 'Self-confidence': 16, 'Enthusiasm': 17, 'Teamwork': 18, 'Goal': 19, 'Critical thinking': 20, 'Flexibility': 21, 'Integrity': 22, 'Hospitality': 23, 'Conflict management': 24, 'Ethic': 25, 'Negotiation': 26, 'Adaptive': 27, 'Active listening': 28, 'Presentation': 29, 'Problem solving': 30, 'Emotional intelligence': 31, 'Argumentation': 32, 'Persuasion': 33, 'Mentoring': 34, 'Writing': 35, 'Decision': 36, 'Accountability': 37, 'Detail': 38, 'Strategic thinking': 39}
{'no skill': 0, 'finance': 1, 'marketing': 2, 'construction, maintenance and transport': 3, 'software engineering': 4, 'logistics': 5, 'design': 

In [144]:
skill_split_layer1 = data_layer1['candidate_skill'].str.split(' ', expand=True)
skill_split_layer2 = data_layer2['candidate_skill'].str.split(' ', expand=True)
skill_split_layer1 = skill_split_layer1.add_prefix('token_')
skill_split_layer2 = skill_split_layer2.add_prefix('token_')

data_layer1 = skill_split_layer1.join(data_layer1)
data_layer2 = skill_split_layer2.join(data_layer2)

data_layer1['token_0'] = data_layer1['token_0'] + ':' + data_layer1['pseudoLabels'].astype(str)
data_layer1['token_1'] = data_layer1['token_1'] + ':' + data_layer1['pseudoLabels'].astype(str)
data_layer1['token_2'] = data_layer1['token_2'] + ':' + data_layer1['pseudoLabels'].astype(str)
data_layer1['token_3'] = data_layer1['token_3'] + ':' + data_layer1['pseudoLabels'].astype(str)

data_layer2['token_0'] = data_layer2['token_0'] + ':' + data_layer2['pseudoLabels'].astype(str)
data_layer2['token_1'] = data_layer2['token_1'] + ':' + data_layer2['pseudoLabels'].astype(str)
data_layer2['token_2'] = data_layer2['token_2'] + ':' + data_layer2['pseudoLabels'].astype(str)
data_layer2['token_3'] = data_layer2['token_3'] + ':' + data_layer2['pseudoLabels'].astype(str)

data_layer1 = data_layer1.fillna('')
data_layer2 = data_layer2.fillna('')

data_layer1['combined'] = data_layer1['left_context'] + ' ' + data_layer1['token_0'] + ' ' + data_layer1['token_1'] + ' ' + data_layer1['token_2'] + ' ' +data_layer1['token_3'] + ' ' + data_layer1['right_context'] + '.'
data_layer2['combined'] = data_layer2['left_context'] + ' ' + data_layer2['token_0'] + ' ' + data_layer2['token_1'] + ' ' + data_layer2['token_2'] + ' ' +data_layer2['token_3'] + ' ' + data_layer2['right_context'] + '.'

data_layer1

,token_0,token_1,token_2,token_3,token_4,origin_sentence,left_context,candidate_skill,right_context,sentiment,...,92.2,93.2,94.2,95.2,96.2,97.2,98.2,99.2,pseudoLabels,combined
0,social:0,medium:0,tool:0,,,this includes educating and recommending socia...,include educate recommend,social medium tool,technology program activity position client,no skill,...,0.062610,-0.901445,0.119658,-2.231398,-0.026215,0.968264,0.625995,-0.502683,0,include educate recommend social:0 medium:0 to...
1,economic:1,,,,,depending on the background and professional e...,attribution benchmarke analysis advanced degree,economic,finance related quantitative field solid,hard skill,...,-0.562210,-0.193270,-0.121037,-0.881090,-0.195065,1.244360,-1.031470,-1.175754,1,attribution benchmarke analysis advanced degre...
2,online:2,marketing:2,,,,our executive team possesses decades of online...,executive team possess decade,online marketing,experience deep understanding digital analytic,hard skill,...,-1.082756,-0.611106,0.671305,-2.038949,-0.838076,1.162073,-0.260606,-0.599322,2,executive team possess decade online:2 marketi...
3,advertising:2,,,,,understand mechanics and intricacies of advert...,understand mechanic intricacy,advertising,ecosystem,hard skill,...,-0.580672,-1.114572,0.917981,-1.008365,-0.186413,-0.172627,0.458130,0.085958,2,understand mechanic intricacy advertising:2 ...
4,clean:3,,,,,this may include cleaning washing and dressing...,include,clean,washing dressing provide meal maintain,hard skill,...,0.950690,-0.750322,1.003237,-0.246654,-0.033974,0.730912,-0.684607,-0.025193,3,include clean:3 washing dressing provide me...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29707,marketing:2,,,,,work with the marketing team to assist in the ...,marketing team assist development product,marketing,plan material,-,...,0.489482,-2.090913,0.040841,0.199304,-0.941954,3.088337,0.447779,-0.593014,2,marketing team assist development product mark...
29708,software:4,development:4,,,,experience with agile software development,experience agile,software development,,-,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4,experience agile software:4 development:4 .
29709,online:2,advertising:2,,,,strong working knowledge of online advertising,strong work knowledge,online advertising,,-,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,strong work knowledge online:2 advertising:2 .
29710,advertising:2,,,,,experience in the advertising space,experience,advertising,space,-,...,-0.925695,0.080737,-0.011589,-1.486976,-0.015590,-1.302662,0.007577,-1.045131,2,experience advertising:2 space.


In [145]:
data_layer1_shuffled = shuffle(data_layer1, random_state=123)
data_layer2_shuffled = shuffle(data_layer2, random_state=123)
data_layer1_shuffled

,token_0,token_1,token_2,token_3,token_4,origin_sentence,left_context,candidate_skill,right_context,sentiment,...,92.2,93.2,94.2,95.2,96.2,97.2,98.2,99.2,pseudoLabels,combined
18432,marketing:2,,,,,sound understanding of the principles of marke...,sound understanding principle,marketing,,-,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,sound understanding principle marketing:2 .
11773,ability:15,to:15,prioritize:15,,,,skill a well a the prove,ability to prioritize,multiple project while maintain attention,soft skill,...,-0.308194,-0.597135,0.995124,-1.007464,0.462078,0.974650,0.812480,0.294888,15,skill a well a the prove ability:15 to:15 pri...
21695,sql:4,server:4,,,,background or aptitude in c sql server softwar...,background aptitude c,sql server,software progamming background,-,...,-0.400426,0.265665,0.463743,-1.794733,-0.553238,1.736874,-0.098119,-0.717703,4,background aptitude c sql:4 server:4 softwar...
17592,clean:0,,,,,driver must have a cdl a and a clean driving r...,driver cdl,clean,driving record,-,...,-0.611333,-0.187969,0.359372,0.279516,1.005442,2.487403,0.843813,0.850819,0,driver cdl clean:0 driving record.
25655,oracle:4,,,,,write java and php code based on requirements ...,project team build maintain support,oracle,database development qa test production,-,...,0.748386,-1.712495,-0.060341,-1.060310,-0.714603,1.476333,-0.622971,0.106865,4,project team build maintain support oracle:4 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28636,html5:4,,,,,development experience with html5 css,development experience,html5,css,-,...,-2.942741,-1.173161,-2.013495,2.021407,-1.364368,-0.640359,-0.695487,0.553439,4,development experience html5:4 css.
17730,financial:1,management:1,,,,we are looking for an experienced financial co...,experienced financial controller undertake aspect,financial management,include corporate accounting regulatory financial,-,...,-0.157077,-1.802988,0.470942,-0.522337,0.428332,1.657672,-0.963494,-0.252798,1,experienced financial controller undertake asp...
28030,inside:2,sale:2,,,,we are looking for an inside sales rep who wil...,help contribute continue growth company,inside sale,rep help generate incremental sale,-,...,-0.104724,-0.152700,0.636948,-1.097827,-0.032135,1.944885,-1.452359,-0.431056,2,help contribute continue growth company inside...
15725,forecast:1,,,,,develop and execute sales business plans to ac...,objective provide accurate timely report,forecast,,-,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,objective provide accurate timely report forec...


In [146]:
print(len(data_layer1_shuffled))
train_layer1 = data_layer1_shuffled[['combined']].iloc[0:round(0.8*len(data_layer1_shuffled)),]
val_layer1 = data_layer1_shuffled[['combined']].drop(train_layer1.index).iloc[0:round(0.1*len(data_layer1_shuffled)),]
test_layer1 = data_layer1_shuffled[['combined']].drop(train_layer1.index).drop(val_layer1.index).iloc[0:round(0.1*len(data_layer1_shuffled)),]
print(len(train_layer1))
print(len(val_layer1))
print(len(test_layer1))
train_layer2 = data_layer2_shuffled[['combined']].iloc[0:round(0.8*len(data_layer2_shuffled)),]
val_layer2 = data_layer2_shuffled[['combined']].drop(train_layer2.index).iloc[0:round(0.1*len(data_layer2_shuffled)),]
test_layer2 = data_layer2_shuffled[['combined']].drop(train_layer2.index).drop(val_layer2.index).iloc[0:round(0.1*len(data_layer2_shuffled)),]

test_layer1 = data_layer1_shuffled[['combined']].sample(frac=0.2,random_state=123)
train_layer1 = data_layer1_shuffled.drop(test_layer1.index)
train_layer1 = train_layer1[['combined']]

test_layer2 = data_layer2_shuffled[['combined']].sample(frac=0.2,random_state=123)
train_layer2 = data_layer2_shuffled.drop(test_layer2.index)
train_layer2 = train_layer2[['combined']]

print(train_layer1)

29712
23770
2971
2971
                                                combined
18432     sound understanding principle marketing:2    .
11773  skill a well a the prove  ability:15 to:15 pri...
21695  background aptitude c sql:4 server:4   softwar...
17592              driver cdl clean:0    driving record.
25655  project team build maintain support oracle:4  ...
...                                                  ...
28636             development experience html5:4    css.
17730  experienced financial controller undertake asp...
28030  help contribute continue growth company inside...
15725  objective provide accurate timely report forec...
19966  server experience build deploy code linux:4   ...

[23770 rows x 1 columns]


In [147]:
def transform(dataset):
    token_df = pd.DataFrame()

    dataset= dataset

    for index in range(0,len(dataset['combined'])):
        tokens = pd.DataFrame(word_tokenize(dataset['combined'].iloc[index]),columns=['tokens'])
        tokens['index'] = index

        token_df = pd.concat([token_df,tokens])

    annotationcolumn = token_df['tokens'].str.split(':',expand=True)

    token_df['annotation'] = annotationcolumn[1]
    token_df['tokens'] = annotationcolumn[0]

    token_df['annotation'] = token_df['annotation'].fillna('0')

    token_df = token_df.rename(columns={'index':'sentence'})

    token_df['tuples'] = list(zip(token_df.tokens, token_df.annotation))

    tuple_list = []
    sublist=[]
    for row in token_df['tuples']:
        sublist.append(row)
        if row[0]=='.':
            tuple_list.append(sublist)
            sublist=[]
            next
    #return sublist
    return tuple_list

training_layer1 = transform(train_layer1)
validation_layer1 = transform(val_layer1)
testing_layer1 = transform(test_layer1)

training_layer2 = transform(train_layer2)
validation_layer2 = transform(val_layer2)
testing_layer2 = transform(test_layer2)

print(training_layer1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [148]:
def make_lists(dataset):
    texts = []
    tags = []

    token_sublist = []
    label_sublist = []

    for sublist in dataset:
        for token,label in sublist:

            token_sublist.append(token)
            label_sublist.append(label)
            if token == '.':
                texts.append(token_sublist)
                tags.append(label_sublist)

                token_sublist = []
                label_sublist = []
    return texts, tags

training_text_layer1, training_label_layer1 = make_lists(training_layer1)
val_text_layer1, val_label_layer1 = make_lists(validation_layer1)
test_text_layer1, test_label_layer1 = make_lists(testing_layer1)

training_text_layer2, training_label_layer2 = make_lists(training_layer2)
val_text_layer2, val_label_layer2 = make_lists(validation_layer2)
test_text_layer2, test_label_layer2 = make_lists(testing_layer2)

print(training_text_layer1)
print(training_label_layer1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [149]:
def create_dict(text, label):
    from collections import defaultdict
    my_dict = defaultdict(list)
    for x in text:
        my_dict['sentences'].append(x)

    for x in label:
        my_dict['tags'].append(x)

    return dict(my_dict)


training_dict_layer1 = create_dict(training_text_layer1, training_label_layer1)
validation_dict_layer1 = create_dict(val_text_layer1, val_label_layer1)
testing_dict_layer1 = create_dict(test_text_layer1, test_label_layer1)

training_dict_layer2 = create_dict(training_text_layer2, training_label_layer2)
validation_dict_layer2 = create_dict(val_text_layer2, val_label_layer2)
testing_dict_layer2 = create_dict(test_text_layer2, test_label_layer2)
print(training_dict_layer1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [154]:
print(list(data_layer1_ddict.values())[1:])
print(list(data_layer2_ddict.values())[1:])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]


In [56]:
from NERDA.models import NERDA
#[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]
tag_scheme_layer1 = list(data_layer1_ddict.values())[1:]
tag_scheme_layer2 = list(data_layer2_ddict.values())[1:]
model_layer1 = NERDA(dataset_training = training_dict_layer1,
              dataset_validation = validation_dict_layer1,
              tag_scheme = tag_scheme_layer1,
              tag_outside = '0',
              transformer = 'bert-base-uncased',
              hyperparameters = {'epochs' : 3,
                                 'warmup_steps' : 500,
                                 'train_batch_size': 13,
                                 'learning_rate': 0.0001},)

model_layer2 = NERDA(dataset_training = training_dict_layer2,
              dataset_validation = validation_dict_layer2,
              tag_scheme = tag_scheme_layer2,
              tag_outside = '0',
              transformer = 'bert-base-uncased',
              hyperparameters = {'epochs' : 3,
                                 'warmup_steps' : 500,
                                 'train_batch_size': 13,
                                 'learning_rate': 0.0001},)
model_layer1_multi = NERDA(dataset_training = training_dict_layer1,
              dataset_validation = validation_dict_layer1,
              tag_scheme = tag_scheme_layer1,
              tag_outside = '0',
              transformer = 'bert-base-multilingual-uncased',
              hyperparameters = {'epochs' : 3,
                                 'warmup_steps' : 500,
                                 'train_batch_size': 13,
                                 'learning_rate': 0.0001},)

model_layer2_multi = NERDA(dataset_training = training_dict_layer2,
              dataset_validation = validation_dict_layer2,
              tag_scheme = tag_scheme_layer2,
              tag_outside = '0',
              transformer = 'bert-base-multilingual-uncased',
              hyperparameters = {'epochs' : 3,
                                 'warmup_steps' : 500,
                                 'train_batch_size': 13,
                                 'learning_rate': 0.0001},)

Device automatically set to: cpu


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Device automatically set to: cpu


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Device automatically set to: cpu


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Device automatically set to: cpu


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [57]:
model_layer1.train()
model_layer1.save_network('Network_Layer1')
model_layer2.train()
model_layer2.save_network('Network_Layer2')
model_layer1_multi.train()
model_layer1_multi.save_network('Network_Layer1_Multi')
model_layer2_multi.train()
model_layer2_multi.save_network('Network_Layer2_Multi')


 Epoch 1 / 3


  1%|▏         | 26/1829 [01:34<1:49:21,  3.64s/it]


KeyboardInterrupt: 

In [65]:
model_layer1.load_network_from_file('./Network_Layer1')
model_layer2.load_network_from_file('./Network_Layer2')
model_layer1_multi.load_network_from_file('./Network_Layer1_Multi')
model_layer2_multi.load_network_from_file('./Network_Layer2_Multi')

'Weights for network loaded from ./Network_Layer2_Multi'

In [87]:
per_1 = model_layer1.evaluate_performance(testing_dict_layer1)
per_2 = model_layer2.evaluate_performance(testing_dict_layer2)
per_1_multi = model_layer1_multi.evaluate_performance(testing_dict_layer1)
per_2_multi = model_layer2_multi.evaluate_performance(testing_dict_layer2)

In [155]:
data_layer1_ddict = {value:key for key, value in data_layer1_ddict.items()}
data_layer2_ddict = {value:key for key, value in data_layer2_ddict.items()}
print(data_layer1_ddict)
print(data_layer2_ddict)

{0: 'no skill', 1: 'business administration', 2: 'education, sales and marketing', 3: 'engineering, construction and transport', 4: 'information technology', 5: 'health and social care', 6: 'science and research', 7: 'Communication', 8: 'Time management', 9: 'Decision making', 10: 'Optimism', 11: 'Coaching', 12: 'Initiative', 13: 'Social skills', 14: 'Creativity', 15: 'Leadership', 16: 'Self-confidence', 17: 'Enthusiasm', 18: 'Teamwork', 19: 'Goal', 20: 'Critical thinking', 21: 'Flexibility', 22: 'Integrity', 23: 'Hospitality', 24: 'Conflict management', 25: 'Ethic', 26: 'Negotiation', 27: 'Adaptive', 28: 'Active listening', 29: 'Presentation', 30: 'Problem solving', 31: 'Emotional intelligence', 32: 'Argumentation', 33: 'Persuasion', 34: 'Mentoring', 35: 'Writing', 36: 'Decision', 37: 'Accountability', 38: 'Detail', 39: 'Strategic thinking'}
{0: 'no skill', 1: 'finance', 2: 'marketing', 3: 'construction, maintenance and transport', 4: 'software engineering', 5: 'logistics', 6: 'design

In [118]:
per_1_copy = per_1.copy()
per_2_copy = per_2.copy()
per_1_multi_copy = per_1_multi.copy()
per_2_multi_copy = per_2_multi.copy()

In [114]:
 print(per_1_copy['Level'][-2:])
per_1_copy['Level'][-2:] = 'AVG_MICRO'
print(per_1_copy)

0    AVG_MICRO
0    AVG_MICRO
Name: Level, dtype: object
        Level  F1-Score  Precision    Recall
0           1  0.805085   0.760000  0.855856
1           2  0.884885   0.873518  0.896552
2           3  0.747664   0.714286  0.784314
3           4  0.867841   0.904856  0.833736
4           5  0.821782   0.855670  0.790476
5           6  0.586207   0.708333  0.500000
6           7  0.941748   0.932692  0.950980
7           8  0.898204   0.842697  0.961538
8           9  0.846154   0.814815  0.880000
9          10  0.866995   0.846154  0.888889
10         11  0.833333   0.769231  0.909091
11         12  0.580645   0.900000  0.428571
12         13  0.846512   0.928571  0.777778
13         14  0.835341   0.764706  0.920354
14         15  0.772908   0.782258  0.763780
15         16  0.825397   1.000000  0.702703
16         17  0.898551   0.892086  0.905109
17         18  0.948276   0.948276  0.948276
18         19  0.831169   0.820513  0.842105
19         20  0.560000   0.466667  0.70000

C:\Users\99mor\AppData\Local\Temp/ipykernel_7160/1357379877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_1_copy['Level'][-2:] = 'AVG_MICRO'


In [119]:
per_1_copy['Level'] = per_1_copy['Level'][:-2].map(lambda x: data_layer1_ddict[x])
per_2_copy['Level'] = per_2_copy['Level'][:-2].map(lambda x: data_layer2_ddict[x])
per_1_multi_copy['Level'] = per_1_multi_copy['Level'][:-2].map(lambda x: data_layer1_ddict[x])
per_2_multi_copy['Level'] = per_2_multi_copy['Level'][:-2].map(lambda x: data_layer2_ddict[x])

per_1_copy['Level'][-2:] = 'AVG_MICRO'
per_2_copy['Level'][-2:] = 'AVG_MICRO'
per_1_multi_copy['Level'][-2:] = 'AVG_MICRO'
per_2_multi_copy['Level'][-2:] = 'AVG_MICRO'


C:\Users\99mor\AppData\Local\Temp/ipykernel_7160/1890729708.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_1_copy['Level'][-2:] = 'AVG_MICRO'
C:\Users\99mor\AppData\Local\Temp/ipykernel_7160/1890729708.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_2_copy['Level'][-2:] = 'AVG_MICRO'
C:\Users\99mor\AppData\Local\Temp/ipykernel_7160/1890729708.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_1_multi_copy['Level']

In [121]:
print('Layer 1 Results')
print(per_1_copy)


Layer 1 Results
                                      Level  F1-Score  Precision    Recall
0                   business administration  0.805085   0.760000  0.855856
1            education, sales and marketing  0.884885   0.873518  0.896552
2   engineering, construction and transport  0.747664   0.714286  0.784314
3                    information technology  0.867841   0.904856  0.833736
4                    health and social care  0.821782   0.855670  0.790476
5                      science and research  0.586207   0.708333  0.500000
6                             Communication  0.941748   0.932692  0.950980
7                           Time management  0.898204   0.842697  0.961538
8                           Decision making  0.846154   0.814815  0.880000
9                                  Optimism  0.866995   0.846154  0.888889
10                                 Coaching  0.833333   0.769231  0.909091
11                               Initiative  0.580645   0.900000  0.428571
12       

In [122]:
print('Layer 2 Results')
print(per_2_copy)

Layer 2 Results
                                               Level  F1-Score  Precision  \
0                                            finance  0.740331   0.690722   
1                                          marketing  0.820253   0.771429   
2            construction, maintenance and transport  0.666667   0.666667   
3                               software engineering  0.873463   0.894535   
4                                          logistics  0.801980   0.771429   
5                                             design  0.894737   0.873288   
6                             administration and law  0.776119   0.764706   
7                                  pr and journalism  0.885496   0.828571   
8                          healthcare administration  0.842105   0.888889   
9                       civil engineering and design  0.730769   0.703704   
10                                      primary care  0.625000   0.625000   
11                      education, languages and art  0.9012

In [123]:
print('Layer 1 Results Multi Lang Bert')
print(per_1_multi_copy)

Layer 1 Results Multi Lang Bert
                                      Level  F1-Score  Precision    Recall
0                   business administration  0.798722   0.757576  0.844595
1            education, sales and marketing  0.885336   0.872477  0.898580
2   engineering, construction and transport  0.754177   0.734884  0.774510
3                    information technology  0.873418   0.892429  0.855200
4                    health and social care  0.787879   0.838710  0.742857
5                      science and research  0.562500   0.600000  0.529412
6                             Communication  0.950495   0.960000  0.941176
7                           Time management  0.892857   0.833333  0.961538
8                           Decision making  0.808511   0.863636  0.760000
9                                  Optimism  0.825688   0.756303  0.909091
10                                 Coaching  0.714286   0.750000  0.681818
11                               Initiative  0.791667   0.703704  0.

In [124]:
print('Layer 2 Results Multi Lang Bert')
print(per_2_multi_copy)

Layer 2 Results Multi Lang Bert
                                               Level  F1-Score  Precision  \
0                                            finance  0.774566   0.752809   
1                                          marketing  0.820126   0.767059   
2            construction, maintenance and transport  0.626866   0.617647   
3                               software engineering  0.880903   0.888199   
4                                          logistics  0.796117   0.752294   
5                                             design  0.890845   0.872414   
6                             administration and law  0.784615   0.796875   
7                                  pr and journalism  0.858238   0.805755   
8                          healthcare administration  0.526316   0.555556   
9                       civil engineering and design  0.763636   0.700000   
10                                      primary care  0.666667   0.714286   
11                      education, languages

In [126]:
per_1_copy.to_csv('per_1_results')
per_2_copy.to_csv('per_2_results')
per_1_multi_copy.to_csv('per_1_multi_results')
per_2_multi_copy.to_csv('per_2_multi_results')

                                      Level  F1-Score  Precision    Recall
0                   business administration  0.805085   0.760000  0.855856
1            education, sales and marketing  0.884885   0.873518  0.896552
2   engineering, construction and transport  0.747664   0.714286  0.784314
3                    information technology  0.867841   0.904856  0.833736
4                    health and social care  0.821782   0.855670  0.790476
5                      science and research  0.586207   0.708333  0.500000
6                             Communication  0.941748   0.932692  0.950980
7                           Time management  0.898204   0.842697  0.961538
8                           Decision making  0.846154   0.814815  0.880000
9                                  Optimism  0.866995   0.846154  0.888889
10                                 Coaching  0.833333   0.769231  0.909091
11                               Initiative  0.580645   0.900000  0.428571
12                       

In [128]:
model_layer1.load_network_from_file('Network_Layer1_deep')
model_layer2.load_network_from_file('Network_Layer2_deep')

per_1_deep = model_layer1.evaluate_performance(testing_dict_layer1)
per_2_deep = model_layer2.evaluate_performance(testing_dict_layer2)

per_1_deep_copy = per_1_deep.copy()
per_2_deep_copy = per_2_deep.copy()

per_1_deep_copy['Level'] = per_1_deep_copy['Level'][:-2].map(lambda x: data_layer1_ddict[x])
per_2_deep_copy['Level'] = per_2_deep_copy['Level'][:-2].map(lambda x: data_layer2_ddict[x])

per_1_deep_copy['Level'][-2:] = 'AVG_MICRO'
per_2_deep_copy['Level'][-2:] = 'AVG_MICRO'

C:\Users\99mor\AppData\Local\Temp/ipykernel_7160/1672018454.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_1_deep_copy['Level'][-2:] = 'AVG_MICRO'
C:\Users\99mor\AppData\Local\Temp/ipykernel_7160/1672018454.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_2_deep_copy['Level'][-2:] = 'AVG_MICRO'


In [129]:
print('Layer 1 Deep Results ')
print(per_1_deep_copy)

Layer 1 Deep Results 
                                      Level  F1-Score  Precision    Recall
0                   business administration  0.799107   0.792035  0.806306
1            education, sales and marketing  0.875346   0.869435  0.881339
2   engineering, construction and transport  0.763990   0.758454  0.769608
3                    information technology  0.864559   0.873496  0.855804
4                    health and social care  0.811881   0.845361  0.780952
5                      science and research  0.580645   0.642857  0.529412
6                             Communication  0.922330   0.913462  0.931373
7                           Time management  0.920245   0.882353  0.961538
8                           Decision making  0.897959   0.916667  0.880000
9                                  Optimism  0.885714   0.837838  0.939394
10                                 Coaching  0.816327   0.740741  0.909091
11                               Initiative  0.810811   0.937500  0.714286
12 

In [130]:
print('Layer 2 Deep Results ')
print(per_2_deep_copy)

Layer 2 Deep Results 
                                               Level  F1-Score  Precision  \
0                                            finance  0.737500   0.776316   
1                                          marketing  0.792746   0.761194   
2            construction, maintenance and transport  0.636364   0.636364   
3                               software engineering  0.863064   0.860965   
4                                          logistics  0.772277   0.742857   
5                                             design  0.877442   0.866667   
6                             administration and law  0.751880   0.746269   
7                                  pr and journalism  0.841699   0.795620   
8                          healthcare administration  0.666667   0.750000   
9                       civil engineering and design  0.701754   0.625000   
10                                      primary care  0.705882   0.666667   
11                      education, languages and art  

In [140]:
import os

layer = 'Layer 2'
transformer = 'bert-base-uncased'
file = '95% pseudolabels - fasttext - layer2 - cleaned'
deep = 'deep'

if os.path.exists("C:/Users/99mor/PycharmProjects/MRP1_Models/Results/" + layer + " " + transformer + " " + deep + ".txt"):
    os.remove("C:/Users/99mor/PycharmProjects/MRP1_Models/Results/" + layer + " " + transformer + " " + deep + ".txt")
with open("C:/Users/99mor/PycharmProjects/MRP1_Models/Results/" + layer + " " + transformer + " " + deep + ".txt", 'w') as f:
    f.write(f"Dataset : '{file}'\n")
    f.write(f"Labels : {layer}\n")
    f.write(f"Transformer : {transformer}\n")
    f.write(f"Depth : {deep}\n")
    f.write(f"Number of training instances : {len(training_layer2)}\n")
    f.write(per_2_deep_copy.to_string())